In [4]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd

In [5]:
# Private credential information has been removed
credentials = service_account.Credentials.from_service_account_info(
    {
  "type": "service_account",
  "project_id": "dhutchings",
  "private_key_id": "XXXXXXXXXXXXXX",
  "private_key": "XXXXXXXXXXXXXXXXXX",
  "client_email": "XXXXXXXXXXXXXXXXXXX",
  "client_id": "XXXXXXXXXXXXXXXXXXXXX",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "XXXXXXXXXXXXXXXXX"
}
)

In [6]:
client = bigquery.Client('dhutchings', credentials = credentials)

In [8]:
# Returns a datatable that shows important metrics on a monthly basis
query = '''
SELECT month, SUM(conversions) AS total_conversions, SUM(impressions) as total_impressions,
SUM(publisher_split) as total_cost,
SUM(conversion_value) AS total_conversion_rev,
(SUM(conversion_value)/SUM(impressions))*1000 AS rev_per_1000_impression, 
(SUM(publisher_split)/SUM(impressions))*1000 AS cost_per_1000_impression,
SUM(impressions)/SUM(conversions) AS impressions_1_conversion,
SUM(referrals)/SUM(conversions) AS referrals_1_conversion
FROM `dhutchings.lockerdome.all_months`
GROUP BY month
'''
df = client.query(query).to_dataframe()
df.to_csv('/tmp/metric_table.csv')

In [10]:
##### Queries below were used to create advertising analytics dashboard #####

In [12]:
# Returns a datatable that shows important metrics on a daily basis
cumulative_query = '''
SELECT date, SUM(conversions) AS total_conversions, SUM(impressions) as total_impressions,
SUM(publisher_split) as total_cost,
SUM(conversion_value) AS total_conversion_rev,
(SUM(conversion_value)/SUM(impressions))*1000 AS rev_per_1000_impression, 
(SUM(publisher_split)/SUM(impressions))*1000 AS cost_per_1000_impression,
SUM(impressions)/SUM(conversions) AS impressions_1_conversion,
SUM(referrals)/SUM(conversions) AS referrals_1_conversion
FROM `dhutchings.lockerdome.all_months`
GROUP BY date
'''
df = client.query(cumulative_query).to_dataframe()
df.to_csv('/tmp/cum_table.csv')

In [14]:
# This query returns key metrics grouped by date and device
device_query = '''
SELECT date, device, SUM(conversions) AS total_conversions, SUM(impressions) as total_impressions,
SUM(publisher_split) as total_cost,
SUM(conversion_value) AS total_conversion_rev,
(SUM(conversion_value)/SUM(impressions))*1000 AS rev_per_1000_impression, 
(SUM(publisher_split)/SUM(impressions))*1000 AS cost_per_1000_impression,
FROM `dhutchings.lockerdome.all_months`
GROUP BY date, device
HAVING device != "Other"
'''
df = client.query(device_query).to_dataframe()
df.to_csv('/tmp/device_table.csv')

In [15]:
# Returns top 10 creative ids based on conversion value per mille
creative_query = '''
SELECT creative_id, 
(SUM(conversion_value)/SUM(impressions))*1000 AS rev_per_1000_impression
FROM `dhutchings.lockerdome.all_months`
GROUP BY creative_id
ORDER BY rev_per_1000_impression DESC
LIMIT 10
'''
df = client.query(creative_query).to_dataframe()
df.to_csv('/tmp/creative_table.csv')

In [16]:
# Returns top 10 ad unit ids based on conversion value per mille
ad_unit_query = '''
SELECT ad_unit_id, 
CASE 
  WHEN SUM(impressions) = 0 THEN 0
  ELSE (SUM(conversion_value)/SUM(impressions))*1000
END AS rev_per_1000_impression
FROM `dhutchings.lockerdome.all_months`
GROUP BY ad_unit_id
ORDER BY rev_per_1000_impression DESC
LIMIT 10
'''
df = client.query(ad_unit_query).to_dataframe()
df.to_csv('/tmp/ad_unit.csv')